# Signe Language Recognition 

Lucas Piquet / Henri Vasseur 

<h1><a id='toc'></a>Sommaire</h1>

<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ul>
        <li><a href="#0">0.  MediaPipe's functions </a></li>
        <li><a href="#I">1. Data Collection </a></li>
        <li><a href="#II">2.  Data Selection / Validity of acquisition</a></li> 
        <li><a href="#III">3. Model Training</a></li>
        <li><a href="#IV">4. Real time test</a></li> 
    </ul>

In [18]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.utils import to_categorical
from scipy import stats
import pygame

<h2><a id='0'></a>
0.  MediaPipe's functions </h2> 

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [3]:
#
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
# 
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [5]:
#
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 


In [6]:
# Extract Keypoint Values
def extract_keypoints(results):
    """Returns:
    - numpy.ndarray:
      A 1D numpy array containing concatenated keypoint values.
      The order of keypoints is as follows:
      [left_hand_landmarks, right_hand_landmarks, pose_landmarks, face_landmarks] 
      shape = (1662,)
    """
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh, rh, pose, face])

<h2><a id='I'></a>
1.  Data Collection </h2> 

In [75]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('Data25') 

# Actions that we try to detect
actions = np.array(['bonjour', 'aurevoir', 'cava'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 25


In [74]:
# Add new folders if needed
# Check if the directory is empty
if not DATA_PATH in os.listdir() :
    # Directory is empty, handle it as needed
    os.makedirs(os.path.join(DATA_PATH))
    # Your code to handle an empty directory goes here
for action in actions:
        
    # Directory is not empty, proceed with creating new directories
    dir_action = os.path.join(DATA_PATH, action)
    
    if not action in os.listdir(DATA_PATH) :
    # Directory is empty, handle it as needed
        os.makedirs(os.path.join(dir_action))
    # Your code to handle an empty directory goes here




In [76]:
# Folder start
start_folder = len(os.listdir(os.path.join(DATA_PATH, actions[0]))) + 1

print(start_folder)
print(actions)


31
['bonjour' 'aurevoir' 'cava']


In [79]:
time.sleep(1)

cap = cv2.VideoCapture(0)

pygame.mixer.init()

# Charger des sons 
start_sound = pygame.mixer.Sound("Sound/start_bip.wav")
end_sound = pygame.mixer.Sound("Sound/end_bip.wav")
action_bip = pygame.mixer.Sound("Sound/action_bip.wav")


# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos

        # Jouer un son pour différencier les actions
        pygame.mixer.Sound.play(action_bip)

        time.sleep(1)

        for sequence in range(start_folder, start_folder + no_sequences):

            # Jouer un son pour indiquer le début de l'enregistrement de la vidéo
            pygame.mixer.Sound.play(start_sound)

            # Initialize results array
            sign = np.zeros((1662,sequence_length))

            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)

                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(1000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                        
                sign[:,frame_num] = extract_keypoints(results)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                
            # Jouer un son pour indiquer la fin de l'enregistrement de la vidéo
            pygame.mixer.Sound.play(end_sound)

            # Save keypoints
            npy_path = os.path.join(DATA_PATH, action, str(sequence))
            np.save(npy_path, sign)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
            
            # Pause between videos
            time.sleep(1.25)

        if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
        pygame.mixer.Sound.play(action_bip)
        time.sleep(1.25)
        
        
    # Update folder start number
    start_folder += no_sequences
    cap.release()
    cv2.destroyAllWindows()

In [9]:
DATA_PATH = os.path.join('Data_viz_test')

In [22]:
# Fast Video Capture

time.sleep(1)
cap = cv2.VideoCapture(0)
pygame.mixer.init()

# Charger un son (remplacez le chemin avec votre propre fichier audio)
start_sound = pygame.mixer.Sound("start_bip.wav")
end_sound = pygame.mixer.Sound("end_bip.wav")

# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # Action to capture (adjust as needed)
    action_to_capture = "hello"

    # Number of frames to capture
    num_frames_to_capture = 30

    # Create an array to store landmarks
    sign_landmarks = np.zeros((1662, num_frames_to_capture))
    
    # Jouer un son au début
    pygame.mixer.Sound.play(start_sound)

    # Capture frames
    for frame_num in range(num_frames_to_capture):
        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)

        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Apply wait logic
        if frame_num == 0:
            cv2.putText(image, 'STARTING COLLECTION', (120, 200),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4, cv2.LINE_AA)
            cv2.putText(image, 'Collecting frames for {} Sign'.format(action_to_capture), (15, 12),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
            # Show to screen
            cv2.imshow('OpenCV Feed', image)
            cv2.waitKey(100)
        else:
            cv2.putText(image, 'Collecting frames for {} Sign'.format(action_to_capture), (15, 12),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
            # Show to screen
            cv2.imshow('OpenCV Feed', image)

        # Store landmarks
        sign_landmarks[:,frame_num] = extract_keypoints(results)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    pygame.mixer.Sound.play(end_sound)
    # Save landmarks
    npy_path = os.path.join(DATA_PATH, action_to_capture)
    np.save(npy_path, sign_landmarks)
    

    # Release resources
    cap.release()
    cv2.destroyAllWindows()


<h2><a id='II'></a>
2.  Data Selection / Validity of acquisition </h2> 

In [82]:
actions = os.listdir(os.path.join(DATA_PATH))
label_map = {label:num for num, label in enumerate(actions)}
nbr_actions = len(actions)

In [83]:
print(label_map)
print(nbr_actions)

{'aurevoir': 0, 'bonjour': 1, 'cava': 2, 'hello': 3, 'iloveyou': 4, 'nogesture': 5, 'thanks': 6}
7


In [84]:
sign = np.load(os.path.join(DATA_PATH, actions[0], "1.npy"))
print(sign.shape)

(1662, 25)


In [85]:
nbr_sign_per_action = {action : len(os.listdir(os.path.join(DATA_PATH, action))) for action in actions}
print(nbr_sign_per_action)
nbr_tot_sign = sum(nbr_sign_per_action.values())
print(nbr_tot_sign)

{'aurevoir': 120, 'bonjour': 120, 'cava': 120, 'hello': 120, 'iloveyou': 120, 'nogesture': 120, 'thanks': 120}
840


In [86]:
# nombre de video dans chaque répertoire 
nbr_tot_sign = sum(nbr_sign_per_action.values())



X = np.zeros((nbr_tot_sign, sequence_length, sign.shape[0]))
y = np.zeros((nbr_tot_sign, 1))

X_without_face = np.zeros((nbr_tot_sign, sequence_length, sign.shape[0]-468*3))

# ...
sum_sign = 0
for action in actions:
    action_path = os.path.join(DATA_PATH, action)
    nbr_sign = nbr_sign_per_action[action]
    

    for sequence, sequence_path in enumerate(os.listdir(action_path)):
        sequence_path = os.path.join(action_path, "{}.npy".format(sequence+1))
        
        # Vérifiez si le fichier existe avant de le charger
        if os.path.exists(sequence_path):
            X[sum_sign+sequence, :, :] = np.load(sequence_path).T
            y[sum_sign+sequence,] = label_map[action]
            X_without_face[sum_sign+sequence, :, :] = np.load(sequence_path).T[:, :-468*3]
        else:
            # print(f"Le fichier {file_path} n'existe pas. Ignorer cette séquence.")
            break  # Arrête la boucle si un fichier est manquant
    sum_sign += nbr_sign

# ...
y = to_categorical(y).astype(int)


In [44]:
print(X.shape, X_without_face.shape,y.shape)

(570, 25, 1662) (570, 25, 258) (570, 7)


In [87]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
y_test

array([[0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 0],
       [1,

In [88]:
X_train_hands, X_test_hands, y_train_hands, y_test_hands = train_test_split(X_without_face, y, test_size=0.2)
print(X_train_hands.shape, X_test_hands.shape, y_train_hands.shape, y_test_hands.shape) #(48, 25, 258) (12, 25, 258) (48, 1) (12, 1)

(672, 25, 258) (168, 25, 258) (672, 7) (168, 7)


<h2><a id='III'></a>
3.  Model Training </h2> 

In [22]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [67]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(sequence_length,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(nbr_actions, activation='softmax'))

In [68]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [69]:
model.fit(X_train, y_train, epochs=500, callbacks=[tb_callback])

Epoch 1/500
17/17 [==============================] - 5s 96ms/step - loss: 2.1813 - categorical_accuracy: 0.2359
Epoch 2/500
17/17 [==============================] - 1s 62ms/step - loss: 1.7712 - categorical_accuracy: 0.2690
Epoch 3/500
17/17 [==============================] - 1s 63ms/step - loss: 1.3407 - categorical_accuracy: 0.5478
Epoch 4/500
17/17 [==============================] - 1s 61ms/step - loss: 2.0850 - categorical_accuracy: 0.3138
Epoch 5/500
17/17 [==============================] - 1s 61ms/step - loss: 1.3493 - categorical_accuracy: 0.4854
Epoch 6/500
17/17 [==============================] - 1s 61ms/step - loss: 1.0720 - categorical_accuracy: 0.5984
Epoch 7/500
17/17 [==============================] - 1s 61ms/step - loss: 0.8241 - categorical_accuracy: 0.6413
Epoch 8/500
17/17 [==============================] - 1s 65ms/step - loss: 0.8704 - categorical_accuracy: 0.6628
Epoch 9/500
17/17 [==============================] - 1s 64ms/step - loss: 0.7634 - categorical_accuracy:

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 4)                 132       
                                                                 
Total params: 596,708
Trainable params: 596,708
Non-trai

In [70]:
# Définir le chemin du répertoire des poids
weights_dir = "Weights"

# Assurez-vous que le répertoire existe, sinon, créez-le
if not os.path.exists(weights_dir):
    os.makedirs(weights_dir)

# Sauvegarder les poids dans le fichier 'action.h5' dans le répertoire des poids
model.save(os.path.join(weights_dir, 'action_recognition_9.h5'))

In [22]:
# Définir le chemin du répertoire des poids
weights_dir = "Weights"
model.load_weights(os.path.join(weights_dir, 'action_recognition.h5'))

### Hand Model Training

In [14]:
model_hands = Sequential()
model_hands.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(sequence_length,258))) # (25, 258)
model_hands.add(LSTM(128, return_sequences=True, activation='relu'))
model_hands.add(LSTM(64, return_sequences=False, activation='relu'))
model_hands.add(Dense(64, activation='relu'))
model_hands.add(Dense(32, activation='relu'))
model_hands.add(Dense(nbr_actions, activation='softmax'))

In [15]:
model_hands.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [28]:
model_hands.fit(X_train_hands, y_train_hands, epochs=700, callbacks=[tb_callback])
# (48, 25, 258) (48, 1)


Epoch 1/700
12/12 [==============================] - 3s 56ms/step - loss: 0.9642 - categorical_accuracy: 0.6042
Epoch 2/700
12/12 [==============================] - 0s 31ms/step - loss: 0.5699 - categorical_accuracy: 0.7396
Epoch 3/700
12/12 [==============================] - 0s 31ms/step - loss: 0.3967 - categorical_accuracy: 0.7812
Epoch 4/700
12/12 [==============================] - 0s 31ms/step - loss: 0.3308 - categorical_accuracy: 0.8203
Epoch 5/700
12/12 [==============================] - 0s 32ms/step - loss: 0.3256 - categorical_accuracy: 0.8333
Epoch 6/700
12/12 [==============================] - 0s 31ms/step - loss: 0.3749 - categorical_accuracy: 0.7943
Epoch 7/700
12/12 [==============================] - 0s 31ms/step - loss: 0.3684 - categorical_accuracy: 0.8203
Epoch 8/700
12/12 [==============================] - 0s 32ms/step - loss: 0.3206 - categorical_accuracy: 0.8542
Epoch 9/700
12/12 [==============================] - 0s 33ms/step - loss: 0.3579 - categorical_accuracy:

In [29]:
model_hands.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 25, 64)            82688     
_________________________________________________________________
lstm_1 (LSTM)                (None, 25, 128)           98816     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 132       
Total params: 237,284
Trainable params: 237,284
Non-trainable params: 0
__________________________________________________

In [16]:
# Définir le chemin du répertoire des poids
weights_dir_hands = "Weights_hands"


In [31]:
# Sauvargarde des poids
# Assurez-vous que le répertoire existe, sinon, créez-le
if not os.path.exists(weights_dir_hands):
    os.makedirs(weights_dir_hands)

# Sauvegarder les poids dans le fichier 'action.h5' dans le répertoire des poids
model_hands.save(os.path.join(weights_dir_hands, 'action_recognition_7.h5'))

In [17]:
# Chargement des poids
model_hands.load_weights(os.path.join(weights_dir_hands, 'action_recognition_7.h5'))

In [89]:
def build_transformer_model(input_shape, num_classes):
    inputs = tf.keras.Input(shape=input_shape)

    # Positional Encoding
    position = layers.Embedding(input_dim=input_shape[0], output_dim=input_shape[1])(tf.range(start=0, limit=input_shape[0], delta=1))

    x = inputs + position

    # Multi-Head Self Attention
    x = layers.MultiHeadAttention(num_heads=8, key_dim=64, dropout=0.1)(x, x)

    # Feed Forward Network
    x = layers.Conv1D(filters=512, kernel_size=1, activation='relu')(x)
    x = layers.Dropout(0.1)(x)
    x = layers.Conv1D(filters=256, kernel_size=1, activation='relu')(x)
    x = layers.GlobalAveragePooling1D()(x)

    # Output layer
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.1)(x)
    x = layers.Dense(num_classes, activation='softmax')(x)

    model = tf.keras.Model(inputs=inputs, outputs=x, name='transformer_model_hands')
    return model

# Specify the input shape and number of classes
input_shape = (sequence_length, 1662)  # Adjust based on your input data
num_classes = nbr_actions  # Adjust based on the number of gesture classes

# Build the Transformer model
transformer_model_hands = build_transformer_model(input_shape, num_classes)

# Compile the model
transformer_model_hands.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
transformer_model_hands.summary()


Model: "transformer_model_hands"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 25, 1662)]   0                                            
__________________________________________________________________________________________________
tf.__operators__.add_4 (TFOpLam (None, 25, 1662)     0           input_6[0][0]                    
__________________________________________________________________________________________________
multi_head_attention_4 (MultiHe (None, 25, 1662)     3406974     tf.__operators__.add_4[0][0]     
                                                                 tf.__operators__.add_4[0][0]     
__________________________________________________________________________________________________
conv1d_8 (Conv1D)               (None, 25, 512)      851456      multi_head_

In [90]:
transformer_model_hands.fit(X_train, y_train, epochs=250, callbacks=[tb_callback])

Epoch 1/250
24/24 [==============================] - 7s 247ms/step - loss: 1.9545 - accuracy: 0.2196
Epoch 2/250
24/24 [==============================] - 6s 241ms/step - loss: 1.0332 - accuracy: 0.5714
Epoch 3/250
24/24 [==============================] - 6s 244ms/step - loss: 0.8018 - accuracy: 0.6548
Epoch 4/250
24/24 [==============================] - 6s 247ms/step - loss: 0.6681 - accuracy: 0.7077
Epoch 5/250
24/24 [==============================] - 7s 292ms/step - loss: 0.5278 - accuracy: 0.7804
Epoch 6/250
24/24 [==============================] - 11s 474ms/step - loss: 0.5671 - accuracy: 0.7884
Epoch 7/250
24/24 [==============================] - 11s 460ms/step - loss: 0.4673 - accuracy: 0.8280
Epoch 8/250
24/24 [==============================] - 11s 443ms/step - loss: 0.6123 - accuracy: 0.7884
Epoch 9/250
24/24 [==============================] - 11s 451ms/step - loss: 0.4779 - accuracy: 0.8214
Epoch 10/250
24/24 [==============================] - 11s 454ms/step - loss: 0.4121 - a

In [91]:
# Définir le chemin du répertoire des poids
weights_dir_hands_transformer = "Weights_hands_transformer"

# Sauvargarde des poids
# Assurez-vous que le répertoire existe, sinon, créez-le
if not os.path.exists(weights_dir_hands_transformer):
    os.makedirs(weights_dir_hands_transformer)

# Sauvegarder les poids dans le fichier 'action.h5' dans le répertoire des poids
transformer_model_hands.save(os.path.join(weights_dir_hands_transformer, 'action_recognition_3.h5'))

In [ ]:
# Définir le chemin du répertoire des poids
weights_dir_hands_transformer = "Weights_hands_transformer"
# Chargement des poids
transformer_model_hands.load_weights(os.path.join(weights_dir_hands_transformer, 'action_recognition_3.h5'))

<h2><a id='IV'></a>
4.  Real time test </h2> 

In [54]:
print(actions)

['aurevoir', 'bonjour', 'cava', 'hello', 'iloveyou', 'nogesture', 'thanks']


In [92]:
colors = [(245,117,16), (117,245,16), (16,117,245), (147, 112, 219), (112, 219, 147), (219, 147, 112), (219, 112, 147)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*70), (int(prob*100), 90+num*70), colors[num], -1)    # (0, 60-300), (0-100, 90-330)
        cv2.putText(output_frame, actions[num], (0, 85+num*70), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

### Hand + Face

In [71]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.9

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-25:]

        
        
        if len(sequence) == 25:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

### Real time Hand test

In [27]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.9

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        
        keypoints = extract_keypoints(results) # (1662,)
        # Add the hand's landmarks to the sequence's list
        sequence.append(keypoints[:258]) # (258,)
        # Control that the length of the sequence is less or equal than 25
        sequence = sequence[-25:]
        
        if len(sequence) == 25: # D'après la ligne précédente ce sera toutjours le cas 
            # renvoie la probabilité d'appartenance à chaque classe sous forme d'une liste contenant un tableau de 4 éléments d'où le [0]
            res = model_hands.predict(np.expand_dims(sequence, axis=0))[0] # (1, 4)
            # print(actions[np.argmax(res)])
            # ajouter à la liste predictions l'indice de la classe ayant la plus grande probabilité
            predictions.append(np.argmax(res))
        
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res):
                # Accept the prediction if the probability is greater than the threshold 
                if res[np.argmax(res)] > threshold: 
                    
                    # Check if the prediction is the same as the previous one
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            # Limite the sentence's length to the last 5 words
            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)

        #Draw the blue rectangle on the top of the screen for the sentence    
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        
        # Draw the sentence
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

## Real time test for the transformer model 

In [93]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.9

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        
        keypoints = extract_keypoints(results) # (1662,)
        # Add the hand's landmarks to the sequence's list
        sequence.append(keypoints) # (258,)
        # Control that the length of the sequence is less or equal than 25
        sequence = sequence[-25:]
        
        if len(sequence) == 25: # D'après la ligne précédente ce sera toutjours le cas 
            # renvoie la probabilité d'appartenance à chaque classe sous forme d'une liste contenant un tableau de 4 éléments d'où le [0]
            res = transformer_model_hands.predict(np.expand_dims(sequence, axis=0))[0] # (1, 4)
            # print(actions[np.argmax(res)])
            # ajouter à la liste predictions l'indice de la classe ayant la plus grande probabilité
            predictions.append(np.argmax(res))
        
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res):
                # Accept the prediction if the probability is greater than the threshold 
                if res[np.argmax(res)] > threshold: 
                    
                    # Check if the prediction is the same as the previous one
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            # Limite the sentence's length to the last 5 words
            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)

        #Draw the blue rectangle on the top of the screen for the sentence    
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        
        # Draw the sentence
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti